### This notebook is for text mining literatures related to our gene mutations

In [16]:
import requests
import sys
import os
import numpy as np
import Bio
import pandas as pd
import json

pd.set_option('display.max_colwidth', -1)

In [18]:
#read in file chr3_annotated_exon-splice.txt and 
data = pd.read_csv("variant_burden/chr3_annotated_exon-splice.txt", sep='\t', keep_default_na=False, na_values=['.'])

data_nonsynonymous = data.loc[data['ExonicFunc.ensGene'] != 'synonymous SNV']
data_nonsynonymous.dropna(thresh=2)


listoftop10nonsynonymous = data_nonsynonymous['hgnc_symbol'].value_counts()
#print (type(listoftop10nonsynonymous))
listoftop10nonsynonymous = listoftop10nonsynonymous.sort_index()
listoftop10nonsynonymous= listoftop10nonsynonymous.nlargest(10)
#print(listoftop10nonsynonymous)

listoftop10nonsynonymous = listoftop10nonsynonymous.index.values.tolist()

#print (listoftop10nonsynonymous)


##################### get list of nonsynonymous gene ##############
df_gene = pd.DataFrame()
list_snps_1 = []

for gene in listoftop10nonsynonymous:
    list_snp_1 = data_nonsynonymous.loc[data_nonsynonymous['hgnc_symbol'] == gene]['avsnp150']
    #deduplicate
    list_snp_1 = list(set(list_snp_1))
    str_snp = ",".join(list_snp_1)
    
    list_snps_1.append(str_snp)

#print (list_snps)

df_gene_list_1 = pd.DataFrame({
                    'gene': listoftop10nonsynonymous,
                    'listofsnp': list_snps_1
                        })

df_gene_list_1




,gene,listofsnp
0,MUC4,"rs2259102,rs2453138,rs882605,rs2246901,rs9882363,rs62282486,rs1106502,rs2293232,rs62282480,rs2177336,rs1104760,rs202060675,rs2641776,rs3107764,rs2259292,rs74507812,rs729593,rs2550240,rs374495657,rs430037,rs13095016"
1,SLC9C1,"rs9809384,rs9809404,rs6768523,rs73853324,rs74840030,rs4434123,rs9288938,rs6781844,rs28516377,rs9872691,rs9828502"
2,CAND2,"rs3732678,rs11718898,rs3732675,rs2305398,rs2305397,rs12629133"
3,COL6A5,"rs12488457,rs819085,rs16827497,rs322117,rs1497312,rs9883988"
4,OR5H8,"rs13096400,rs2316270,rs13076085,rs2316269,rs4325958,rs2316271"
5,CFAP44,"rs2291905,rs2270781,rs59722850,rs810209,rs4682484"
6,DNAH12,"rs4681982,rs6773904,rs6806444,rs4462937,rs4060726"
7,FYCO1,"rs13079478,rs13059238,rs3796375,rs4683158,rs3733100"
8,HRG,"rs2228243,rs1042445,rs9898,rs10770,rs1042464"
9,NAALADL2,"rs9836841,rs9823911,rs9866564,rs9826737,rs6802937"


In [19]:
data_uncommon = data_nonsynonymous.loc[data_nonsynonymous['PopFreqMax'] < 0.05]

listoftop10uncommon = data_uncommon['hgnc_symbol'].value_counts()
listoftop10uncommon = listoftop10uncommon.sort_index()
listoftop10uncommon= listoftop10uncommon.nlargest(10)
#print (listoftop10uncommon)

listoftop10uncommon = listoftop10uncommon.index.values.tolist()

####################### get list of uncommon gene ###############
df_gene = pd.DataFrame()
list_snps_2 = []

for gene in listoftop10uncommon:
    list_snp_2 = data_nonsynonymous.loc[data_nonsynonymous['hgnc_symbol'] == gene]['avsnp150']
    #deduplicate
    list_snp_2 = list(set(list_snp_2))
    str_snp = ",".join(list_snp_2)
    
    list_snps_2.append(str_snp)

#print (list_snps)

df_gene_list_2 = pd.DataFrame({
                    'gene': listoftop10uncommon,
                    'listofsnp': list_snps_2
                        })

df_gene_list_2

,gene,listofsnp
0,ACAA1,rs2229528
1,ACTRT3,"rs2068178,rs34273285"
2,ALDH1L1,"rs202042780,rs2886059,rs2276724,rs4646750"
3,CMSS1,rs11537817
4,COMMD2,"rs371197410,rs9843784"
5,GLB1,"rs4302331,rs35289681,rs7637099"
6,IFT122,rs781181264
7,IGSF10,"rs6440742,rs34114908,rs7619322"
8,MAGI1,rs149895880
9,MFN1,"rs7637065,rs143476739"


In [26]:
import os

for gene in df_gene_list_1['gene']:
    f= open("ns/" + gene + ".txt", "w+")
    
    
    # snp list
    gene_mu_string = df_gene_list_1[df_gene_list_1['gene']==gene]["listofsnp"]
    print (gene_mu_string.values[0])
    
    
    f.write("Related Key Words Count for Gene: " + gene + "\n")
    f.write("==========================================================================\n")

    
    for snp in gene_mu_string.values[0].split(","):
        f.write("\n")
        f.write("SNP: " + snp + "\n")
        # LitVar API to fetch related key words related to searching rsid ('gene','variant','chemical','drug')
        searchURLCOUNT = "https://www.ncbi.nlm.nih.gov/research/bionlp/litvar/api/v1/public/relations"
        paramsCOUNT = {}

        data_string = '{"accessions": ["litvar@' + snp + '##"], "unlimited": 0}'
        data_COUNT = json.loads(data_string)

        req = requests.post(searchURLCOUNT, params=paramsCOUNT, json=data_COUNT)

        # return value as json of {rsid,pmids}
        return_data = req.json()

        #print (return_data)

        for i in range(len(return_data)):
            data = return_data[i]
            f.write ("Category: " + data['concept'] + "\n")

            relations = data['relations']
            f.write ("--------------------->\n")
            for j in range(len(relations)):
                #print (relations[j])
                f.write ("\t" + relations[j]['name'] + "\t" + str(relations[j]['count']) + "\n")
    
    f.write("\n\n")
    f.write("Related literatures for Gene: " + gene + "\n")
    f.write("==========================================================================\n")

    # LitVar API to fetch related literatures for gene mutations
    searchURL = "https://www.ncbi.nlm.nih.gov/research/bionlp/litvar/api/v1/public/rsids2pmids"
    
    params={'rsids':gene_mu_string} 
    req=requests.get(searchURL,params=params)
    related_lit = []

    # return value as json of {rsid,pmids}
    return_data = req.json()

    for item in return_data:
        related_lit.append(item['pmids'])
        #print (item['pmids'])
    #print (item["rsid"])

    # PubMed API to fetch metadata of literature by given pmid
    from Bio.Entrez import efetch
    from Bio import Entrez
    Entrez.email = 'guannan.gong@yale.edu'

    for lit in related_lit:
        lit_string = ",".join(str(e) for e in lit)
        #print (lit_string)
        handle = efetch(db='pubmed', id=lit_string, retmode='text', rettype='abstract')
        f.write (handle.read())
        f.write("----------------------------------------------------------------------------\n")
    f.close()


rs2259102,rs2453138,rs882605,rs2246901,rs9882363,rs62282486,rs1106502,rs2293232,rs62282480,rs2177336,rs1104760,rs202060675,rs2641776,rs3107764,rs2259292,rs74507812,rs729593,rs2550240,rs374495657,rs430037,rs13095016
rs9809384,rs9809404,rs6768523,rs73853324,rs74840030,rs4434123,rs9288938,rs6781844,rs28516377,rs9872691,rs9828502
rs3732678,rs11718898,rs3732675,rs2305398,rs2305397,rs12629133
rs12488457,rs819085,rs16827497,rs322117,rs1497312,rs9883988
rs13096400,rs2316270,rs13076085,rs2316269,rs4325958,rs2316271
rs2291905,rs2270781,rs59722850,rs810209,rs4682484
rs4681982,rs6773904,rs6806444,rs4462937,rs4060726
rs13079478,rs13059238,rs3796375,rs4683158,rs3733100
rs2228243,rs1042445,rs9898,rs10770,rs1042464
rs9836841,rs9823911,rs9866564,rs9826737,rs6802937


In [29]:
for gene in df_gene_list_2['gene']:
    f= open("ns_uncommon/" + gene + ".txt", "w+")
    
    
    # snp list
    gene_mu_string = df_gene_list_2.loc[df_gene_list_2['gene']==gene]["listofsnp"]
    print (gene_mu_string.values[0])
    
    f.write("Related Key Words Count for Gene: " + gene + "\n")
    f.write("==========================================================================\n")

    
    for snp in gene_mu_string.values[0].split(","):
        f.write("\n")
        f.write("SNP: " + snp + "\n")
        searchURLCOUNT = "https://www.ncbi.nlm.nih.gov/research/bionlp/litvar/api/v1/public/relations"
        paramsCOUNT = {}
        #print (snp)

        data_string = '{"accessions": ["litvar@' + snp + '##"], "unlimited": 0}'
        data_COUNT = json.loads(data_string)

        req = requests.post(searchURLCOUNT, params=paramsCOUNT, json=data_COUNT)

        # return value as json of {rsid,pmids}
        return_data = req.json()

        #print (return_data)

        for i in range(len(return_data)):
            data = return_data[i]
            f.write ("Category: " + data['concept'] + "\n")

            relations = data['relations']
            f.write ("--------------------->\n")
            for j in range(len(relations)):
                #print (relations[j])
                f.write ("\t" + relations[j]['name'] + "\t" + str(relations[j]['count']) + "\n")
    
    f.write("\n\n")
    f.write("Related literatures for Gene: " + gene + "\n")
    f.write("==========================================================================\n")

    # LitVar API to fetch related literatures for gene mutations
    searchURL = "https://www.ncbi.nlm.nih.gov/research/bionlp/litvar/api/v1/public/rsids2pmids"
    
    params={'rsids':gene_mu_string} 
    req=requests.get(searchURL,params=params)
    related_lit_2 = []

    # return value as json of {rsid,pmids}
    return_data = req.json()

    for item in return_data:
        related_lit_2.append(item['pmids'])
        #print (item['pmids'])
    #print (item["rsid"])

    # PubMed API to fetch metadata of literature by given pmid
    from Bio.Entrez import efetch
    from Bio import Entrez
    Entrez.email = 'guannan.gong@yale.edu'

    for lit in related_lit_2:
        lit_string = ",".join(str(e) for e in lit)
        #print (lit_string)
        handle = efetch(db='pubmed', id=lit_string, retmode='text', rettype='abstract')
        f.write (handle.read())
        f.write("----------------------------------------------------------------------------\n")
    f.close()
    

rs2229528
rs2068178,rs34273285
rs202042780,rs2886059,rs2276724,rs4646750
rs11537817
rs371197410,rs9843784
rs4302331,rs35289681,rs7637099
rs781181264
rs6440742,rs34114908,rs7619322
rs149895880
rs7637065,rs143476739
